In [7]:
import seaborn as sns
import metapack as mp
import pandas as pd
import geopandas as gpd
import rowgenerators as rg
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [22]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

In [3]:
gdf = gpd.read_file('/Users/eric/Downloads/General_Plan_Land_Use__Los_Angeles/Los_Angeles_PL_SCAG.shp')

In [4]:
gdf.SCAG_GP_CO.value_counts()

1110 Single Family Residential                                                                              1367606
1120 Multi-Family Residential                                                                                308420
1140 Mixed Residential                                                                                        91688
1150 Rural Residential                                                                                        79266
1200 General Commercial                                                                                       39812
1600 Mixed Residential and Commercial                                                                         32545
1220 Retail and Commercial and Services                                                                       31682
1310 Light Industrial                                                                                         17866
1311 Light Manufacturing                                                

In [5]:
gpco = gpd.GeoDataFrame({
    'objectid': gdf.OBJECTID,
    'SCAG_GP_CO': gdf.SCAG_GP_CO,
    'geometry': gdf.geometry.centroid
})



In [8]:
tracts = rg.geoframe('census://2017/5/CA/tract')
gpco.crs = tracts.crs # Just a guess

gptracts = gpd.sjoin(tracts, gpco)

gpt = gptracts.copy()
gpt['code'] = gpt.SCAG_GP_CO.apply(lambda v: int((v.split())[0]) if v else 0)


In [15]:
df = gpt[['geoid','code','OBJECTID']].merge(gdf, on='OBJECTID')
df = df[['geoid','OBJECTID','APN','CITY','code','ZONE_CODE','ACRES']]
df.columns = [c.lower() for c in df.columns]

In [20]:
df = df.merge(gpco[['objectid','geometry']], on='objectid')

In [24]:
!ls ../data/

la-planning-code.csv tract_to_parcels.csv


In [28]:
df.to_csv('../data/tract_to_parcels.csv')


In [27]:
df.index.name = 'index'
df.head()

,geoid,objectid,apn,city,code,zone_code,acres,geometry
index,,,,,,,,
0,14000US06037204920,205663,5191-012-009,Los Angeles,1120,R2,0.119355,POINT (-118.1997972661181 34.02204043785111)
1,14000US06037204920,205653,5191-011-008,Los Angeles,1120,R2,0.123901,POINT (-118.201096570206 34.02206941378159)
2,14000US06037204920,204574,5191-011-022,Los Angeles,1120,R2,0.119314,POINT (-118.2007983464134 34.02207176228492)
3,14000US06037204920,204573,5191-011-015,Los Angeles,1120,R2,0.104612,POINT (-118.1999064954099 34.02208907018797)
4,14000US06037204920,213146,5191-011-014,Los Angeles,1120,R2,0.119352,POINT (-118.2000169669352 34.02213571331608)
